In [42]:
# importing all the necessary libaries
import os
import csv
import time
import string
import keyboard
from re import search
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [23]:
# define your school name right here:

School_name = "Institut National des Postes et Télécommunications"

In [24]:
# defining the webdriver and config btw this code will be almost the same in all of your selenium scripts
options = Options()

# !!! blocking browser notifications !!!
prefs = {"profile.default_content_setting_values.notifications": 2}
options.add_experimental_option("prefs", prefs)

# starting in maximized window
options.add_argument("start-maximized")
options.add_argument("--disable-default-apps")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\tahae\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [25]:
driver.get("https://www.linkedin.com/login/")

In [26]:
username = os.environ.get("ln_us")
password = os.environ.get("ln_ps")

In [27]:
username_field = driver.find_element(By.ID,"username")
username_field.send_keys(username)

password_field = driver.find_element(By.ID,"password")
password_field.send_keys(password)

password_field.submit()

In [28]:
'''search_bar = driver.find_element(By.CLASS_NAME,"search-global-typeahead__input")
search_bar.send_keys(School_name)
time.sleep(1)
keyboard.press_and_release('enter')'''

'search_bar = driver.find_element(By.CLASS_NAME,"search-global-typeahead__input")\nsearch_bar.send_keys(School_name)\ntime.sleep(1)\nkeyboard.press_and_release(\'enter\')'

In [29]:
driver.get("https://www.linkedin.com/search/results/schools/?keywords="+School_name)
time.sleep(3)

In [30]:
first_result = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, """//*[@id="main"]/div/div/div[1]/ul/li/div/div/div[2]/div/div[1]/div/span/span/a""")))
first_result.click()

In [34]:
driver.get(driver.current_url+"people")

In [35]:
count = 0
items_urls = []
testing = []
cards_links = driver.find_elements(By.CLASS_NAME, "org-people-profile-card__profile-info")
print(len(cards_links))
for card in cards_links: print(card.text+"\n----------------------------------\n")


12
Wiaam Sahraoui
2nd degree connection
· 2nd
Data Engineer | Trading Process Analyst at Société Générale - Africa Technologies & Services
’21 Data engineering
1 shared connection
----------------------------------

Abdelhamid AABIROUCH
2nd degree connection
· 2nd
Data Scientist | Machine Learning/ Deep Learning Engineer | FreeLancer
’20 management
1 shared connection
----------------------------------

zait chaimae
2nd degree connection
· 2nd
FullSatck Engineer at Flow digital Studio
’20 la telecommunications
1 shared connection
----------------------------------

Oumaima Outamldou
2nd degree connection
· 2nd
Consultante sécurité SI - PCI DSS
’21 Cybersecurity
1 shared connection
----------------------------------

Karima EL-ASEKARY
3rd degree connection
· 3rd
Software Engineer - Digital Factory at OCP SA
----------------------------------

Ilyass Khaouja
3rd degree connection
· 3rd
Manager at PwC - MBA graduate - Fulbright Scholar
----------------------------------

Ahmed Rahouti
3rd

In [36]:
print("lengh is: "+str(len(cards_links)))
for card in cards_links: print(card.text+"\n----------------------------------\n")

lengh is: 12
Wiaam Sahraoui
2nd degree connection
· 2nd
Data Engineer | Trading Process Analyst at Société Générale - Africa Technologies & Services
’21 Data engineering
1 shared connection
----------------------------------

Abdelhamid AABIROUCH
2nd degree connection
· 2nd
Data Scientist | Machine Learning/ Deep Learning Engineer | FreeLancer
’20 management
1 shared connection
----------------------------------

zait chaimae
2nd degree connection
· 2nd
FullSatck Engineer at Flow digital Studio
’20 la telecommunications
1 shared connection
----------------------------------

Oumaima Outamldou
2nd degree connection
· 2nd
Consultante sécurité SI - PCI DSS
’21 Cybersecurity
1 shared connection
----------------------------------

Karima EL-ASEKARY
3rd degree connection
· 3rd
Software Engineer - Digital Factory at OCP SA
----------------------------------

Ilyass Khaouja
3rd degree connection
· 3rd
Manager at PwC - MBA graduate - Fulbright Scholar
----------------------------------

Ahmed R

In [37]:
count = 0
items_urls = []
testing = []

cards = driver.find_elements(By.XPATH, "//body[@class='render-mode-BIGPIPE nav-v2 ember-application icons-loaded boot-complete']/div[@class='application-outlet']/div[@class='authentication-outlet']/div[@class='organization-outlet relative']/div[2]/div[1]/div[2]/main[1]/div[2]/div[1]/div[2]/div[1]/div[1]/ul[1]/li/section[1]/div[1]/div[1]/div[1]/a")

for card in cards:
    print(card.get_attribute('href'))


https://www.linkedin.com/in/wiaam-sahraoui-00a7bb1a9/
https://www.linkedin.com/in/abdelhamid-aabirouch-235528185/
https://www.linkedin.com/in/zait-chaimae-27b640158/
https://www.linkedin.com/in/oumaima-outamldou-686484180/
https://www.linkedin.com/in/karima-el-asekary-a6922b86/
https://www.linkedin.com/in/ikhaouja/
https://www.linkedin.com/in/ahmedrht/
https://www.linkedin.com/in/ahmed-touil-4600432b/
https://www.linkedin.com/in/youssefkhantache/
https://www.linkedin.com/in/abdelaziz-ouaaddi/
https://www.linkedin.com/in/wahib-ouahib-a9b937135/


In [39]:

count = 0
items_urls = []
testing = []
while len(items_urls) < 12:

    # keyboard.press_and_release('down arrow')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/.1);")
    time.sleep(.7)
    cards_links = driver.find_elements(By.XPATH, "//body[@class='render-mode-BIGPIPE nav-v2 ember-application icons-loaded boot-complete']/div[@class='application-outlet']/div[@class='authentication-outlet']/div[@class='organization-outlet relative']/div[2]/div[1]/div[2]/main[1]/div[2]/div[1]/div[2]/div[1]/div[1]/ul[1]/li/section[1]/div[1]/div[1]/div[1]/a")
    if cards_links == testing:
        break
    for card in cards_links:
        try:
            items_urls.append(card.get_attribute('href'))
        except:
            continue
    testing = driver.find_elements(By.XPATH, "//body[@class='render-mode-BIGPIPE nav-v2 ember-application icons-loaded boot-complete']/div[@class='application-outlet']/div[@class='authentication-outlet']/div[@class='organization-outlet relative']/div[2]/div[1]/div[2]/main[1]/div[2]/div[1]/div[2]/div[1]/div[1]/ul[1]/li/section[1]/div[1]/div[1]/div[1]/a")
    cards_links = []
print(items_urls)

['https://www.linkedin.com/in/wiaam-sahraoui-00a7bb1a9/', 'https://www.linkedin.com/in/abdelhamid-aabirouch-235528185/', 'https://www.linkedin.com/in/zait-chaimae-27b640158/', 'https://www.linkedin.com/in/oumaima-outamldou-686484180/', 'https://www.linkedin.com/in/karima-el-asekary-a6922b86/', 'https://www.linkedin.com/in/ikhaouja/', 'https://www.linkedin.com/in/ahmedrht/', 'https://www.linkedin.com/in/ahmed-touil-4600432b/', 'https://www.linkedin.com/in/youssefkhantache/', 'https://www.linkedin.com/in/abdelaziz-ouaaddi/', 'https://www.linkedin.com/in/wahib-ouahib-a9b937135/', 'https://www.linkedin.com/in/yassine-benja%C3%A2-testlead-testautomation/', 'https://www.linkedin.com/in/malak-tahri-706aab10a/', 'https://www.linkedin.com/in/abdellatif-mezzine-b58b5a181/', 'https://www.linkedin.com/in/otman-driouch-95a50547/', 'https://www.linkedin.com/in/mehdi-boumzebra-369bbb146/', 'https://www.linkedin.com/in/khaoula-belhaj-soulami-a0810985/', 'https://www.linkedin.com/in/hajji-abderrahmane/

In [47]:
profilesID = items_urls
names = []
companies = []
locs = []

for profileID in profilesID:
    fulllink =  profileID
    driver.get(fulllink)
    time.sleep(8)
    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')

    #scraping locations
    name_div = soup.find_all('h4', {'class': 'pv-entity__location'})
    loc = []
    for i in range(len(name_div)):
        span = name_div[i].find_all('span')
        loc.append(span[1].get_text().strip())
    locs.append(loc)

    #scraping companies
    name_div = soup.find_all('p', {'class': 'pv-entity__secondary-title t-14 t-black t-normal'})
    company = []
    for i in range(len(name_div)):
        company.append(name_div[i].get_text().strip())
    companies.append(company)

TimeoutException: Message: timeout: Timed out receiving message from renderer: 299.119
  (Session info: chrome=100.0.4896.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x00F47413+2389011]
	Ordinal0 [0x00ED9F61+1941345]
	Ordinal0 [0x00DCC658+837208]
	Ordinal0 [0x00DBDB80+777088]
	Ordinal0 [0x00DBC995+772501]
	Ordinal0 [0x00DBCF98+774040]
	Ordinal0 [0x00DC796F+817519]
	Ordinal0 [0x00DD243B+861243]
	Ordinal0 [0x00DD4E10+871952]
	Ordinal0 [0x00DBD266+774758]
	Ordinal0 [0x00DD2289+860809]
	Ordinal0 [0x00E2464D+1197645]
	Ordinal0 [0x00E13F66+1130342]
	Ordinal0 [0x00DEE546+976198]
	Ordinal0 [0x00DEF456+980054]
	GetHandleVerifier [0x010F9632+1727522]
	GetHandleVerifier [0x011ABA4D+2457661]
	GetHandleVerifier [0x00FDEB81+569713]
	GetHandleVerifier [0x00FDDD76+566118]
	Ordinal0 [0x00EE0B2B+1968939]
	Ordinal0 [0x00EE5988+1989000]
	Ordinal0 [0x00EE5A75+1989237]
	Ordinal0 [0x00EEECB1+2026673]
	BaseThreadInitThunk [0x76006739+25]
	RtlGetFullPathName_UEx [0x77E28E7F+1215]
	RtlGetFullPathName_UEx [0x77E28E4D

In [ ]:
ids = [i for i range(len(items_url))]
df = pd.DataFrame(list(zip(ids, companies, locs)), 
               columns =['id', 'companies', 'locations']) 

In [15]:
while len(items_urls) < 100:

    # keyboard.press_and_release('down arrow')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/.1);")
    time.sleep(.7)
    cards_links = driver.find_elements(By.CLASS_NAME, "org-people-profile-card__profile-info")
    if cards_links == testing:
        count += 1
    if count == 10:
        break
    for card in cards_links:
        try:
            item_url = card.get_attribute('href')
            items_urls.append(item_url)
        except:
            continue
    testing = driver.find_elements_by_xpath(
        "//section[@class='annonces ng-star-inserted']//a")
    cards_links = []
    print(len(items_urls))

C:\Users\tahae\AppData\Local\Temp/ipykernel_19192/2444876976.py:17: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  testing = driver.find_elements_by_xpath(


0
0
0
0
0
0
0
0
0
